<a href="https://colab.research.google.com/github/PyBlin/Study/blob/main/PyML/Chapter2_ScikitLearn/Chap2_5_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.데이터 전처리 (Data Preprocessing)

* GIGO (Garbage In, Garbage Out)

## 5.1 Data Encoding

### 5.1.1 Label Encoding

* 카테고리 피쳐를 코드형 숫자값으로 변환합니다.
* 사이킷런의 레이블 인코딩은 LabelEncoder 클래스로 구현합니다.
* LabelEncoder를 객체로 생성한 후 `fit()`과 `transform()`을 호출해 레이블 인코딩을 수행합니다.

In [1]:
from sklearn.preprocessing import LabelEncoder

items = ['TV', '냉장고', '전자렌지', '컴퓨터', 
         '선풍기', '선풍기', '믹서', '믹서']

# LabelEncoder를 객체로 생성한 후 fit()과 transform()으로 레이블 인코딩 수행
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print(f"인코딩 반환값 : {labels}")

인코딩 반환값 : [0 1 4 5 3 3 2 2]


* TV --> 0
* 냉장고 --> 1
* 전자렌지 --> 4
* 컴퓨터 --> 5
* 선풍기 --> 3
* 믹서 --> 2
    - 문자열이 많은 경우에는 classes_ 속성값으로 확인합니다.

In [2]:
print(f"인코딩 클래스 : {encoder.classes_}")

인코딩 클래스 : ['TV' '냉장고' '믹서' '선풍기' '전자렌지' '컴퓨터']


In [3]:
print(f"디코딩 원본값 : {encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3])}")

디코딩 원본값 : ['전자렌지' '컴퓨터' '믹서' 'TV' '냉장고' '냉장고' '선풍기' '선풍기']


* `inverse_transform()`을 통해 인코딩된 값을 다시 디코딩할 수 있습니다.

### 5.1.2 One-Hot Encoding

* 레이블 인코딩은 선형 회귀와 같은 알고리즘에는 적용하지 않아야합니다.
* 왜냐하면 숫자에 가중치가 부여될 수 있기 때문입니다.
* 원-핫 인코딩은 이러한 문제점을 해결해줍니다.

In [4]:
# OneHotEncoder 클래스로 쉽게 변환 가능
# 변환 전에 : 모든 문자열은 숫자형으로, 입력값은 2차원으로
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

items = ['TV', '냉장고', '전자렌지', '컴퓨터', 
         '선풍기', '선풍기', '믹서', '믹서']

# 숫자값으로 변환을 위해 LabelEncoder로 변환
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

# 2차원 데이터로 변환
labels = labels.reshape(-1, 1)

# One-Hot Encoding 적용
oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)
print(f"One-Hot Encoding Data : \n{oh_labels.toarray()}\n")
print(f"One-Hot Encoding Data Dimension : \n{oh_labels.shape}")

One-Hot Encoding Data : 
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]

One-Hot Encoding Data Dimension : 
(8, 6)


In [5]:
# pandas One-Hot Encoding : get_dummies()
import pandas as pd

df = pd.DataFrame({'item':['TV', '냉장고', '전자렌지', '컴퓨터', 
         '선풍기', '선풍기', '믹서', '믹서']})
pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자렌지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


* 판다스에는 One-Hot Encoding을 더 쉽게 지원하는 API가 있습니다.
* `get_dummies()`를 이용하면 문자열을 숫자형으로 변환할 필요 없이 바로 가능합니다.

## 5.2 Feature Scaling and Normalization

* Feature Scaling : 서로 다른 변수의 값 범위를 일정 수준으로 맞추는 작업입니다.
* 대표적으로 표준화(Standardization)와 정규화(Normalization)가 있습니다.

* 표준화 : 데이터 피처 각각을 평균이 0, 분산이 1인 가우시안 정규 분포를 가진 값으로 변환합니다.
    - x[i]_new = (x[i]-mean(x)) / stdev(x)
        - x[i]_new : 표준화를 통해 변환될 피처 x의 새로운 i번째 데이터
        - x[i] : 원래 피처 x의 값
        - mean(x) : 피처 x의 평균
        - stdev(x) : 피처 x의 표준편차 (standard deviation)

* 정규화 : 서로 다른 피처의 크기를 통일하기 위해 크기를 변환해줍니다. 즉, 개별 데이터의 크기를 모두 똑같은 단위로 변경합니다.
    - x[i]_new = (x[i]-min(x)) / (max(x)-min(x))

* 사이킷런의 Normalizer 모듈 : 개별 벡터의 크기를 맞추기 위해 변환합니다. 즉, 개별 벡터를 모든 피처 벡터의 크기로 나누어줍니다. (선형대수 개념)
    - x[i]_new = x[i] / (sqrt((x[i])^2 + (y[i])^2 + (z[i])^2))

* 혼선 방지를 위해 일반적인 의미의 표준화와 정규화를 피처 스케일링으로 통칭하고, 선형대수 개념의 정규화를 벡터 정규화로 지칭합니다.

### 5.2.1 Standard Scaler

* 표준화를 쉽게 지원하기 위한 클래스입니다.
* 즉, 개별 피처를 평균이 0, 분산이 1인 값으로 변환해줍니다.

In [6]:
from sklearn.datasets import load_iris
import pandas as pd

# 붓꽃 데이터셋을 로딩하고 DataFrame으로 변환
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)

print(f"feature mean : \n{iris_df.mean()}\n")
print(f"feature variance : \n{iris_df.var()}")

feature mean : 
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

feature variance : 
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [7]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
scaler = StandardScaler()

# StandardScaler로 데이터셋 변환 : fit(), transform() 호출
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transfrom()에서 변환된 데이터셋이 ndarray로 반환된 것을 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print(f"feature mean : \n{iris_df_scaled.mean()}\n")
print(f"feature variance : \n{iris_df_scaled.var()}")

feature mean : 
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

feature variance : 
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


* 모든 컬럼값의 평균이 0, 분산이 1에 가까운 값으로 변환되었습니다.

### 5.2.2 Min Max Scaler

* 데이터값을 0 ~ 1 범위값으로 변환합니다. (음수가 있으면 -1 ~ 1)
* 데이터 분포가 가우시안 분포가 아닐 때 적용합니다.

In [8]:
from sklearn.datasets import load_iris
import pandas as pd

# 붓꽃 데이터셋을 로딩하고 DataFrame으로 변환
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)



from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# MinMaxScaler로 데이터셋 변환 : fit(), transform() 호출
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transfrom()에서 변환된 데이터셋이 ndarray로 반환된 것을 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)
print(f"feature minimum : \n{iris_df_scaled.min()}\n")
print(f"feature maximum : \n{iris_df_scaled.max()}")

feature minimum : 
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

feature maximum : 
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


* 모든 컬럼값이 0 ~ 1 범위로 변환되었습니다.

## 5.3 학습/테스트 데이터의 스케일링 변환 시 유의점

* scaler 객체를 이용해 학습 데이터셋으로 `fit()`, `transform()`을 적용하면 테스트 데이터셋으로는 다시 `fit()`을 수행하지 않고 학습 데이터셋으로 `fit()`을 수행한 결과를 이용해 `transform()` 변환을 적용해야 합니다.
* 테스트 데이터에 `fit()`을 적용하면 어떤 문제가 생기는지 알아봅시다.

In [9]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습 데이터는 0 ~ 10, 테스트 데이터는 0 ~ 5
# Scaler 클래스의 fit(), transform()은 2차원 이상의 데이터만 가능
train_array = np.arange(11).reshape(-1, 1)
test_array = np.arange(6).reshape(-1, 1)
print(f"train_array : \n{train_array}\n")
print(f"test_array : \n{test_array}")

train_array : 
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]]

test_array : 
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]]


* 여기에 MinMaxScaler를 이용해 변환해봅시다.
* `fit()`을 적용하면 최솟값 0, 최댓값 10, 1/10 Scale이 적용됩니다.
* `transform()`을 호출하면 1/10 Scale로 학습 데이터를 변환합니다.
* 원본 데이터 : 
    - 1 --> 0.1
    - 2 --> 0.2
    - 5 --> 1

In [10]:
"""MinMaxScaler 객체에 별도의 feature_range 파라미터값을 지정하지 않으면 
0~1 값으로 변환"""
scaler = MinMaxScaler()

# fit() --> train_array 최솟값 0, 최댓값 10
scaler.fit(train_array)

# 1/10 scale로 train_array 데이터 변환
train_scaled = scaler.transform(train_array)

print(f"Original train_array data : \n{np.round(train_array.reshape(-1), 2)}\n")
print(f"Scaled train_array data : \n{np.round(train_scaled.reshape(-1), 2)}")

Original train_array data : 
[ 0  1  2  3  4  5  6  7  8  9 10]

Scaled train_array data : 
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [11]:
# 이번에는 테스트 데이터셋을 변환
scaler.fit(test_array)  # 최솟값 0, 최댓값 5

# 1/5 scale
test_scaled = scaler.transform(test_array)

print(f"Original test_array data : \n{np.round(test_array.reshape(-1), 2)}\n")
print(f"Scaled test_array data : \n{np.round(test_scaled.reshape(-1), 2)}")

Original test_array data : 
[0 1 2 3 4 5]

Scaled test_array data : 
[0.  0.2 0.4 0.6 0.8 1. ]


* 학습 데이터와 테스트 데이터의 스케일링이 맞지 않습니다.
    - 학습 데이터 스케일링 : 0, 0.1, 0.2, ... , 1
    - 테스트 데이터 스케일링 : 0, 0.2, 0.4, 0.6, 0.8, 1
* 이제 정상적으로 적용시켜 봅시다.

In [12]:
scaler = MinMaxScaler()
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)
print(f"Original train_array data : \n{np.round(train_array.reshape(-1), 2)}\n")
print(f"Scaled train_array data : \n{np.round(train_scaled.reshape(-1), 2)}\n")

# test_array에 scale 변환 시 : fit() 호출 금지
test_scaled = scaler.transform(test_array)
print(f"Original test_array data : \n{np.round(test_array.reshape(-1), 2)}\n")
print(f"Scaled test_array data : \n{np.round(test_scaled.reshape(-1), 2)}")

Original train_array data : 
[ 0  1  2  3  4  5  6  7  8  9 10]

Scaled train_array data : 
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

Original test_array data : 
[0 1 2 3 4 5]

Scaled test_array data : 
[0.  0.1 0.2 0.3 0.4 0.5]


* 학습 데이터와 테스트 데이터 모두 1/10 수준으로 스케일링 되었습니다.
* 따라서 모두 동일하게 변환되었습니다.
* `fit_transform()`을 적용할 때도 마찬가지입니다. (테스트 데이터에서는 사용 금지)
* 이러한 주의 사항이 발생하므로 학습용/테스트용 분리 전에 전체 데이터셋에 스케일링을 적용하는 것이 바람직합니다.